# Connect to G-Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
default_dir = "/content/drive/MyDrive/.../Data Preprocessing I"
os.chdir(default_dir)

# Recap: exploring data with pandas

## Hiking Dataset

In [ ]:
import pandas as pd
hiking = pd.read_json("hiking.json")
hiking.head()

In [ ]:
hiking.info()

# Missing Value Handlings

## College Dataset

### Load Dataset

In [ ]:
college = pd.read_csv("college.csv")
college.head()

In [ ]:
college.info()

### Detect Missing Values

In [ ]:
csat_unique = college.csat.unique()

In [ ]:
import numpy as np
np.sort(csat_unique)

### Replace Missing Values with NaN

In [ ]:
college = pd.read_csv("college.csv", na_values='.')
college.head()

In [ ]:
college.info()

## Diabetes Dataset

### Load Dataset

In [ ]:
diabetes = pd.read_csv('pima-indians-diabetes.csv')
diabetes.head()

In [ ]:
diabetes.info()

### Detect Missing Values

In [ ]:
diabetes.describe()

In [ ]:
diabetes.BMI[diabetes.BMI == 0]

### Replace Missing Values with NaN

In [ ]:
diabetes.loc[diabetes.BMI == 0, 'BMI'] = np.nan

#### Re-check if the replacement have been done!

In [ ]:
diabetes.BMI[np.isnan(diabetes.BMI)]

In [ ]:
diabetes.BMI[diabetes.BMI.isna()]

In [ ]:
diabetes.BMI[diabetes.BMI.isnull()]

## Imputations: Basic techniques

### Mean Imputation

In [ ]:
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer(strategy='mean')
diabetes_mean = pd.DataFrame(
    data=mean_imputer.fit_transform(diabetes),
    columns=diabetes.columns
)

In [ ]:
diabetes_mean

In [ ]:
diabetes_mean.info()

### Median Imputation

In [ ]:
median_imputer = SimpleImputer(strategy='median')
diabetes_median = pd.DataFrame(
    data=median_imputer.fit_transform(diabetes),
    columns=diabetes.columns
)

### Mode Imputation

In [ ]:
mode_imputer = SimpleImputer(strategy='most_frequent')
diabetes_mode = pd.DataFrame(
    data=mode_imputer.fit_transform(diabetes),
    columns=diabetes.columns
)

### Imputing a constant

In [ ]:
constant_imputer = SimpleImputer(strategy='constant', fill_value=0)
diabetes_constant = pd.DataFrame(
    data=constant_imputer.fit_transform(diabetes),
    columns=diabetes.columns
)

### Scatterplot of imputation

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
nullity = diabetes['Serum_Insulin'].isnull() + diabetes['Glucose'].isnull()

In [ ]:
diabetes_mean.plot(
    x='Serum_Insulin',
    y='Glucose',
    kind='scatter',
    alpha=0.5,
    c=nullity,
    cmap='rainbow',
    title='Mean Imputation'
);

### Visualizing Imputations

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
nullity = diabetes['Serum_Insulin'].isnull() + diabetes['Glucose'].isnull()
imputations = {
    'Mean Imputation': diabetes_mean,
    'Median Imputation': diabetes_median,
    'Most Frequent Imputation': diabetes_mode,
    'Constant Imputation': diabetes_constant
}

for ax, df_key in zip(axes.flatten(), imputations):
    imputations[df_key].plot(
        x='Serum_Insulin',
        y='Glucose',
        kind='scatter',
        alpha=0.5,
        c=nullity,
        cmap='rainbow',
        ax=ax,
        colorbar=False, title=df_key
    )

# Label Encoder
Encoding Categorical Variables


In [ ]:
users = pd.DataFrame(
    data = {
        'user': [1, 2, 3, 4],
        'subscribed': ['y', 'n', 'n', 'y'],
        'fav_color': ['blue', 'green', 'orange', 'green'],
        'loyalty_badge': ['Gold', 'Silver', 'Silver', 'Platinum']
    })

users

## Encoding Binary Variables

### with Pandas

In [ ]:
users['subscribed']

In [ ]:
users["sub_enc"] = users["subscribed"].apply(lambda val: 1 if val == "y" else 0)

In [ ]:
users[["subscribed", "sub_enc"]]

### with Scikit-learn

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
users["sub_enc_le"] = le.fit_transform(users["subscribed"])

print(users[["subscribed", "sub_enc_le"]])

## Encoding Categorical - Nominal Variable

In [ ]:
le_color = LabelEncoder()
users["color_enc_le"] = le_color.fit_transform(users["fav_color"])

print(users[["fav_color", "color_enc_le"]])

## Encoding Categorical - Ordinal Variable

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Ordered as the category rank
badge_cat = ['Silver', 'Gold', 'Platinum']
oe = OrdinalEncoder(categories=[badge_cat])

users["badge_enc_oe"] = oe.fit_transform(users[["loyalty_badge"]])
users["badge_enc_oe"] = users["badge_enc_oe"].astype('int')

print(users[["loyalty_badge", "badge_enc_oe"]])

# One-Hot Encoding
To Categorical-Nominal Feature

In [ ]:
users["fav_color"]

In [ ]:
pd.get_dummies(users["fav_color"])

In [ ]:
users = pd.get_dummies(users, columns = ["fav_color"])
users

# Standardization

## Log Normalization

In [ ]:
df = pd.DataFrame(
    data={
        "col1":[1, 1.20, 0.75, 1.60],
        "col2":[3, 45.5, 28.0, 100]
    }
)

df

In [ ]:
print(df.var())

In [ ]:
df["log_2"] = np.log(df["col2"])
print(df)

In [ ]:
print(df[['col1', 'log_2']].var())

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df),
    columns=df.columns
)

In [ ]:
print(df_scaled)

In [ ]:
print(df_scaled.var())

# Train-test Splitting
Case: Standardized data and modeling with K-Nearest Neighbor


## Checking Missing Values

In [ ]:
diabetes_median.info()

## Dataset Splitting

In [ ]:
from sklearn.model_selection import train_test_split

X = diabetes_median.iloc[:, :-1]
y = diabetes_median["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

## Modeling with KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

knn = KNeighborsClassifier()
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model Evaluation Before & After Scaling

In [ ]:
knn.fit(X_train, y_train)
print("Accuracy - Before Scaled:", knn.score(X_test, y_test))

In [ ]:
knn.fit(X_train_scaled, y_train)
print("Accuracy - After Scaled:", knn.score(X_test_scaled, y_test))

In [ ]:
(0.753 - 0.6753)/0.6753